In [1]:
import mat73
import pandas as pd
import numpy as np
import scipy.io
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'
    

## Sensor Level

In [3]:
foof = pd.read_csv(DATA_DIR+"/sensor_intercept_slope.csv")
columns = [['Electrode %d - Intercept'%(i), 'Electrode %d - Slope'%(i)] for i in range(1, 106)]
columns = list(np.array(columns).flatten())
columns.append('IDs')
foof.columns = columns
foof

,Electrode 1 - Intercept,Electrode 1 - Slope,Electrode 2 - Intercept,Electrode 2 - Slope,Electrode 3 - Intercept,Electrode 3 - Slope,Electrode 4 - Intercept,Electrode 4 - Slope,Electrode 5 - Intercept,Electrode 5 - Slope,...,Electrode 101 - Slope,Electrode 102 - Intercept,Electrode 102 - Slope,Electrode 103 - Intercept,Electrode 103 - Slope,Electrode 104 - Intercept,Electrode 104 - Slope,Electrode 105 - Intercept,Electrode 105 - Slope,IDs
0,1.610769,1.793923,1.639958,1.803579,1.622763,1.867163,1.596286,1.868653,1.459489,1.880624,...,1.637747,1.472414,1.775967,1.534612,1.803604,1.575800,1.869977,0.986272,1.825774,NDARAA075AMK
1,1.572269,1.762184,1.615888,1.938907,1.515331,1.878222,1.577389,1.888830,1.645799,1.892993,...,1.698141,1.698537,1.854421,1.486611,1.755794,1.517099,1.845983,1.486650,1.888544,NDARAA112DMH
2,1.414507,1.818785,1.504383,1.902537,1.563395,1.936381,1.554663,2.085351,1.566065,2.106747,...,1.847331,1.417519,1.869143,1.323095,1.838333,1.496008,1.895424,1.593155,2.095749,NDARAA117NEJ
3,0.561062,1.068994,0.333554,0.991180,0.555779,1.304169,0.665864,1.491464,0.524828,1.573271,...,1.464311,0.576556,1.265274,0.199181,0.863242,0.246682,1.010393,0.703331,1.724831,NDARAA947ZG5
4,1.262007,1.901401,1.305927,1.924721,1.293914,1.924840,1.226456,1.947274,0.818595,1.905113,...,1.859064,1.158560,1.863190,1.173287,1.773532,1.252168,1.878925,0.918020,1.749441,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,0.648706,1.508826,0.591809,1.640814,0.640782,1.665891,0.455759,1.679019,0.866016,1.786340,...,1.536850,0.528776,1.478037,0.466827,1.767683,0.570274,1.664126,1.351549,1.996940,NDARZN277NR6
2038,1.332278,1.729382,1.387863,1.788208,1.452167,1.841329,1.524229,1.976239,1.480653,2.022584,...,1.876485,1.214977,1.795080,1.456392,1.835807,1.399127,1.795906,1.380795,2.036327,NDARZN578YDP
2039,0.746123,1.324138,0.653335,1.285734,0.564490,1.271931,0.375903,1.245331,0.258508,1.206112,...,1.307891,0.691917,1.310004,0.604437,1.283587,0.564744,1.273098,0.339229,1.050644,NDARZN610GTY
2040,0.976055,1.441830,0.823197,1.407064,0.879368,1.464799,0.902699,1.512205,1.012450,1.505105,...,1.341166,0.598687,1.282644,0.697733,1.421177,0.753861,1.457204,0.781225,1.470061,NDARZN677EYE


In [4]:
data = scipy.io.loadmat(DATA_DIR+'x.mat')  
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))

# sparsing
df = np.array(df).reshape(data['x'].shape)
df_sparsed = np.concatenate([np.expand_dims(df[:,:,i:i+2].mean(axis = 2), axis = 2) for i in range(0, data['x'].shape[2]-2, 2)], axis = 2)
df = pd.DataFrame(df_sparsed.reshape((df_sparsed.shape[0], -1)))

#scaling
norm = MinMaxScaler().fit(df)
df = norm.transform(df)
df = pd.DataFrame(df.reshape((df.shape[0], -1)))

columns = np.asarray([['Electrode %d - %d Hz'%(i+1, j+1)] for i in range(df_sparsed.shape[1]) for j in range(df_sparsed.shape[2])])
df.columns = columns
df['IDs'] = foof['IDs']
df

,"(Electrode 1 - 1 Hz,)","(Electrode 1 - 2 Hz,)","(Electrode 1 - 3 Hz,)","(Electrode 1 - 4 Hz,)","(Electrode 1 - 5 Hz,)","(Electrode 1 - 6 Hz,)","(Electrode 1 - 7 Hz,)","(Electrode 1 - 8 Hz,)","(Electrode 1 - 9 Hz,)","(Electrode 1 - 10 Hz,)",...,"(Electrode 105 - 31 Hz,)","(Electrode 105 - 32 Hz,)","(Electrode 105 - 33 Hz,)","(Electrode 105 - 34 Hz,)","(Electrode 105 - 35 Hz,)","(Electrode 105 - 36 Hz,)","(Electrode 105 - 37 Hz,)","(Electrode 105 - 38 Hz,)","(Electrode 105 - 39 Hz,)",IDs
0,1.212006e-10,2.857691e-08,0.000006,0.001066,0.033290,0.143816,0.221608,0.467715,0.593581,0.401686,...,0.000007,0.000002,4.333844e-07,9.373567e-08,1.616373e-08,3.256580e-09,4.986456e-10,7.457518e-11,1.130004e-11,NDARAA075AMK
1,1.539866e-01,3.856645e-01,0.074284,0.216988,0.177267,0.161251,0.117761,0.132384,0.049898,0.004720,...,0.031636,0.009416,2.352137e-03,4.122513e-04,4.825197e-05,5.944433e-06,5.749150e-07,6.398086e-08,8.415637e-09,NDARAA112DMH
2,3.439263e-05,4.110931e-03,0.094118,0.367821,0.284890,0.164614,0.216934,0.333176,0.526883,0.511715,...,0.000311,0.000079,2.045964e-05,4.443460e-06,7.491343e-07,1.436770e-07,2.039047e-08,2.751924e-09,3.663748e-10,NDARAA117NEJ
3,2.832192e-01,3.613309e-01,0.378969,0.299813,0.247412,0.058588,0.015256,0.004514,0.003988,0.061608,...,0.158691,0.150551,1.580043e-01,1.518115e-01,1.233778e-01,1.242931e-01,1.009636e-01,8.498748e-02,7.690149e-02,NDARAA947ZG5
4,1.247342e-04,1.132987e-02,0.167700,0.368840,0.155464,0.104191,0.194459,0.288490,0.233210,0.125520,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,2.303407e-04,2.867255e-02,0.194945,0.081646,0.018422,0.076959,0.179779,0.300051,0.417245,0.459853,...,0.093399,0.058729,4.012462e-02,2.464919e-02,1.257993e-02,7.816617e-03,3.846395e-03,1.926406e-03,1.018633e-03,NDARZN277NR6
2038,1.385606e-02,2.124126e-02,0.032991,0.043456,0.048827,0.049759,0.074988,0.222227,0.444972,0.464653,...,0.016832,0.009602,5.877454e-03,3.190797e-03,1.418509e-03,7.564419e-04,3.146594e-04,1.311957e-04,5.687030e-05,NDARZN578YDP
2039,1.089647e-02,3.749309e-02,0.139003,0.289797,0.304728,0.196113,0.191950,0.342653,0.413368,0.344024,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARZN610GTY
2040,6.411911e-03,2.687961e-02,0.095551,0.241121,0.434011,0.574767,0.694741,0.815939,0.678021,0.490314,...,0.059230,0.026090,1.169173e-02,4.410850e-03,1.294309e-03,4.329110e-04,1.073561e-04,2.536768e-05,5.924775e-06,NDARZN677EYE


In [5]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
print('Before:'+str(beh.shape))

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders',  'No Diagnosis Given', 'Other Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# removing patients with incomplete eval
initial_size = beh.shape[0]
beh = beh[beh.DX_01 != 'No Diagnosis Given: Incomplete Eval']
beh = beh.reset_index(drop=True)
new_size = beh.shape[0]

print('After:'+str(beh.shape))
print('Removing', initial_size - new_size,
      'patients as their evaluations was incomplete.')

Before:(3076, 177)
After:(2939, 177)
Removing 137 patients as their evaluations was incomplete.


In [6]:
no_diagnosis_given = 'No Diagnosis Given'
diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}
diagnoses_to_ids

{'Attention-Deficit/Hyperactivity Disorder': 0,
 'Anxiety Disorders': 1,
 'Specific Learning Disorder': 2,
 'Autism Spectrum Disorder': 3,
 'Disruptive': 4,
 'Communication Disorder': 5,
 'Depressive Disorders': 6,
 'No Diagnosis Given': 7,
 'Other Disorders': 8}

In [7]:
def get_disorder(data, row, index):
    disorder = data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(beh.shape[0]):
    i = 0
    disorder = get_disorder(beh, k, i)
    disorders_patient = []
    while not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        else:
            if diagnoses_to_ids['Other Disorders'] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids['Other Disorders'])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(beh, k, i)

        
    order_of_disorders.append(disorders_patient)

In [8]:
other_disorders = []
no_diagnosis_given = []
for i in order_of_disorders:
    if 7 in i:
        no_diagnosis_given.append(1)
        i.remove(7)
    else:
        no_diagnosis_given.append(0)
    if 8 in i:
        other_disorders.append(1)
        i.remove(8)
    else:
        other_disorders.append(0)

In [9]:
max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

classes = np.zeros((len(most_common_disorders),
                    beh.shape[0]), dtype=np.int32)

df_disorders = beh[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)

    disorder_df = df_disorders[mask.any(axis=1)]

    np.add.at(classes[i], disorder_df.index.values, 1)

behaviour_data_columns = beh.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = beh.drop(columns=columns_to_drop)

for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)

<ipython-input-9-48b714085ca7>:25: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  behaviour_data_columns = beh.columns.values.astype(np.str)


In [10]:
common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders']

labels=behaviour_data[["IDs"]+list(common_disorders)]
labels["Other Disorders"] = other_disorders
labels

<ipython-input-10-26c912096f42>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["Other Disorders"] = other_disorders


,IDs,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders
0,NDARAA075AMK,0,0,0,0,0,0,0,0
1,NDARAA112DMH,1,0,0,0,1,0,0,1
2,NDARAA117NEJ,1,0,0,0,1,0,0,1
3,NDARAA306NT2,1,1,1,0,0,1,0,1
4,NDARAA504CRN,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
2934,NDARZZ007YMP,0,0,0,1,0,0,0,1
2935,NDARZZ740MLM,1,0,0,0,0,0,0,0
2936,NDARZZ810LVF,0,0,0,1,0,1,0,1
2937,NDARZZ830JM7,0,0,0,1,0,0,0,1


In [11]:
add_features = behaviour_data[['Sex', 'Age','IDs']]
add_features.shape

(2939, 3)

In [12]:
df = pd.merge(df, foof, on='IDs', how='inner')
df = pd.merge(df, labels, on='IDs', how='inner')
df = pd.merge(df, add_features, on='IDs', how='inner')
df

,"(Electrode 1 - 1 Hz,)","(Electrode 1 - 2 Hz,)","(Electrode 1 - 3 Hz,)","(Electrode 1 - 4 Hz,)","(Electrode 1 - 5 Hz,)","(Electrode 1 - 6 Hz,)","(Electrode 1 - 7 Hz,)","(Electrode 1 - 8 Hz,)","(Electrode 1 - 9 Hz,)","(Electrode 1 - 10 Hz,)",...,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders,Sex,Age
0,1.212006e-10,2.857691e-08,0.000006,0.001066,0.033290,0.143816,0.221608,0.467715,0.593581,0.401686,...,0,0,0,0,0,0,0,0,1,6.728040
1,1.539866e-01,3.856645e-01,0.074284,0.216988,0.177267,0.161251,0.117761,0.132384,0.049898,0.004720,...,1,0,0,0,1,0,0,1,0,5.545744
2,3.439263e-05,4.110931e-03,0.094118,0.367821,0.284890,0.164614,0.216934,0.333176,0.526883,0.511715,...,1,0,0,0,1,0,0,1,0,7.475929
3,2.832192e-01,3.613309e-01,0.378969,0.299813,0.247412,0.058588,0.015256,0.004514,0.003988,0.061608,...,1,0,1,1,0,0,0,1,0,13.627880
4,1.247342e-04,1.132987e-02,0.167700,0.368840,0.155464,0.104191,0.194459,0.288490,0.233210,0.125520,...,1,0,0,0,0,0,0,1,1,7.982660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,1.556547e-05,2.900630e-04,0.003548,0.023882,0.088943,0.189421,0.293481,0.445277,0.644915,0.577096,...,0,1,0,0,0,0,0,0,1,11.629363
1917,2.303407e-04,2.867255e-02,0.194945,0.081646,0.018422,0.076959,0.179779,0.300051,0.417245,0.459853,...,1,1,0,0,0,0,1,0,1,14.878736
1918,1.089647e-02,3.749309e-02,0.139003,0.289797,0.304728,0.196113,0.191950,0.342653,0.413368,0.344024,...,0,0,0,0,0,0,0,1,1,16.379534
1919,6.411911e-03,2.687961e-02,0.095551,0.241121,0.434011,0.574767,0.694741,0.815939,0.678021,0.490314,...,1,0,0,0,0,0,0,0,1,15.029545


In [13]:
behaviour_data_reg = pd.read_csv(DATA_DIR+'AllData.csv')
behaviour_data_reg_ = behaviour_data_reg.rename(columns = {'EID': 'IDs'}, inplace = True)
behaviour_data_reg.columns

Index(['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_SR_GD', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum'],
      dtype='object')

In [14]:
behaviour_data_reg = behaviour_data_reg[['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]
behaviour_data_reg.shape

(2579, 9)

In [15]:
df = pd.merge(df, behaviour_data_reg, on='IDs', how='left')
df.shape

(1921, 4324)

In [16]:
disorders_list = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders', 'Other Disorders']

labels_list = [ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum', 'WISC_VCI_Sum']

x_sensor = pd.DataFrame.copy(df)
x_sensor = x_sensor.drop(disorders_list+labels_list+['SWAN_Avg'], axis=1)

y = df[disorders_list+labels_list]
y["IDs"] = x_sensor["IDs"] #adding IDs for split

print(x_sensor.shape, y.shape)


(1921, 4308) (1921, 16)


<ipython-input-16-d8d537e25934>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["IDs"] = x_sensor["IDs"] #adding IDs for split


## Source Level

In [17]:
# ## data loading
# data_source = mat73.loadmat(DATA_DIR+'x_source_new.mat')  
foof_source = pd.read_csv(DATA_DIR+"source_intercept_slope.csv")
foof_source = foof_source[foof_source.columns.difference(['Unnamed: 0'])]
columns = [['Parcel %d - Intercept'%(i), 'Parcel %d - Slope'%(i)] for i in range(1, 69)]
columns = list(np.array(columns).flatten())
columns.append('IDs')
foof_source.columnsns = columns

<ipython-input-17-6535bee4c592>:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  foof_source.columnsns = columns


In [18]:
foof_source

,0,1,10,100,101,102,103,104,105,106,...,91,92,93,94,95,96,97,98,99,IDs
0,0.160123,1.362617,0.326530,0.737488,1.551151,0.640907,1.609206,0.727309,1.335153,0.740876,...,1.455204,0.935387,1.591959,0.775190,1.558807,0.651631,1.552995,0.131833,1.398957,NDARAA075AMK
1,0.988915,1.330636,0.842708,1.744219,1.620000,1.816369,1.665852,1.276499,1.647900,1.143253,...,1.554655,1.752204,1.485544,1.772740,1.583015,1.678093,1.557056,1.465154,1.637982,NDARAA112DMH
2,1.455004,1.515717,1.059826,1.978837,1.800016,1.688798,1.739504,1.891130,1.665696,1.857908,...,1.550177,2.204919,1.795133,2.201063,1.813078,1.082781,1.633553,1.475778,1.618034,NDARAA117NEJ
3,0.796931,1.547997,-0.362581,0.808001,1.218828,0.615191,1.196636,0.342273,1.098845,0.483636,...,1.081387,0.805774,1.121730,0.828743,1.123967,0.127056,1.261766,0.141541,1.089746,NDARAA947ZG5
4,-0.627481,1.123518,-0.183652,0.453893,1.476552,0.257925,1.499167,0.134314,1.305685,0.102059,...,1.315691,0.541945,1.390186,0.455097,1.402555,0.036022,1.194515,-0.231320,1.291719,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,-0.561398,1.060676,-0.531291,-0.148225,1.100029,-0.208995,1.029133,0.209871,1.031523,0.187472,...,1.111754,0.286677,1.165853,0.187645,1.132932,-0.163983,1.135515,-0.427048,1.025244,NDARZN277NR6
2038,1.178082,1.344137,0.717890,1.752047,1.662216,1.924005,1.729488,1.598476,1.585771,1.585412,...,1.615807,2.022862,1.705277,2.021837,1.713112,1.250597,1.535746,1.453465,1.648811,NDARZN578YDP
2039,-0.394953,1.348060,-1.276275,-0.311917,1.275042,-0.479055,1.284871,-0.102165,1.147873,-0.217476,...,1.244733,0.105129,1.350318,-0.160253,1.305165,-0.418728,1.221250,-0.738842,1.259117,NDARZN610GTY
2040,-0.347506,0.870524,0.136223,0.865577,1.211928,0.848283,1.360736,0.750430,0.921142,0.801516,...,1.270069,0.649872,1.061642,0.931554,1.154281,-0.398291,0.965341,0.452792,1.306080,NDARZN677EYE


In [19]:
data = mat73.loadmat(DATA_DIR+'x_source_new.mat')  
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))

# sparsing
df2 = np.array(df2).reshape(data['x'].shape) 
df2_sparsed = np.concatenate([np.expand_dims(df2[:,:,i:i+2].mean(axis = 2), axis = 2) for i in range(0, data['x'].shape[2]-2, 2)], axis = 2)
df2 = pd.DataFrame(df2_sparsed.reshape((df2_sparsed.shape[0], -1)))

#scaling
norm = MinMaxScaler().fit(df2)
df2 = norm.transform(df2)
df2 = pd.DataFrame(df2.reshape((df2.shape[0], -1)))

columns = np.asarray([['Parcel %d - %d Hz'%(i+1, j+1)] for i in range(68) for j in range(39)])
df2.columns = columns
df2['IDs'] = foof['IDs']

df_source = pd.DataFrame.copy(df2)

In [20]:
x_source = pd.merge(df_source, foof_source, on='IDs', how='inner')
x_source = pd.merge(x_source, add_features, on='IDs', how='inner')
x_source.shape

(1921, 2791)

## Split

In [21]:
train_features_sensor, test_features_sensor, train_labels, test_labels = train_test_split(x_sensor, y, test_size=0.25, shuffle=True)

In [22]:
train_features_source = x_source[x_source.IDs.isin(train_features_sensor.IDs)]
test_features_source = x_source[x_source.IDs.isin(test_features_sensor.IDs)]

In [23]:
# classfication
train_labels_classification = train_labels[disorders_list+["IDs"]]
test_labels_classification = test_labels[disorders_list+["IDs"]]

# regression
train_labels_regression = train_labels[labels_list+["IDs"]]
test_labels_regression = test_labels[labels_list+["IDs"]]
test_labels_regression = test_labels_regression.dropna()

In [24]:
# modify test feature set based on column drop
test_features_sensor_regression = test_features_sensor[test_features_sensor.IDs.isin(test_labels_regression.IDs)]
test_features_source_regression = test_features_source[test_features_source.IDs.isin(test_labels_regression.IDs)]

In [25]:
test_features_sensor_classification = pd.DataFrame.copy(test_features_sensor)
test_features_source_classification = pd.DataFrame.copy(test_features_source)

In [26]:
# sanity checks
assert sorted(test_features_source_classification.IDs) == sorted(test_features_sensor_classification.IDs)
assert sorted(test_features_source_regression.IDs) == sorted(test_features_sensor_regression.IDs)
assert sorted(train_features_source.IDs) == sorted(train_features_sensor.IDs)

In [27]:
# saving to csv (hacky fix)

path = os.path.join(DATA_DIR, 'split_data')

#saving
if not(os.path.exists(path)):
    os.mkdir(path)
train_features_source.to_pickle(os.path.join(path, 'train_features_source.pkl'))
test_features_source_classification.to_pickle(os.path.join(path, 'test_features_source_classification.pkl'))
train_features_sensor.to_pickle(os.path.join(path, 'train_features_sensor.pkl'))
test_features_sensor_classification.to_pickle(os.path.join(path, 'test_features_sensor_classification.pkl'))
train_labels_classification.to_pickle(os.path.join(path, 'train_labels_classification.pkl'))
test_labels_classification.to_pickle(os.path.join(path, 'test_labels_classification.pkl'))
test_features_source_regression.to_pickle(os.path.join(path, 'test_features_source_regression.pkl'))
test_features_sensor_regression.to_pickle(os.path.join(path, 'test_features_sensor_regression.pkl'))
train_labels_regression.to_pickle(os.path.join(path, 'train_labels_regression.pkl'))
test_labels_regression.to_pickle(os.path.join(path, 'test_labels_regression.pkl'))

# save to csv
import pickle as pkl
import pandas as pd

new_path = os.path.join(DATA_DIR, 'split_data_csv')
for file in os.listdir(path):
    df = pd.read_pickle(os.path.join(path, file))
    df.to_csv(os.path.join(new_path, file.split('.')[0]+'.csv'), index=False)

In [28]:
#remove IDs
train_features_source = train_features_source[train_features_source.columns.difference(['IDs'])]
test_features_source_classification = test_features_source_classification[test_features_source_classification.columns.difference(['IDs'])]
train_features_sensor = train_features_sensor[train_features_sensor.columns.difference(['IDs'])]
test_features_sensor_classification = test_features_sensor_classification[test_features_sensor_classification.columns.difference(['IDs'])]
test_features_source_regression = test_features_source_regression[test_features_source_regression.columns.difference(['IDs'])]
test_features_sensor_regression = test_features_sensor_regression[test_features_sensor_regression.columns.difference(['IDs'])]
test_labels_regression = test_labels_regression[test_labels_regression.columns.difference(['IDs'])]
train_labels_classification = train_labels_classification[train_labels_classification.columns.difference(['IDs'])]
test_labels_classification = test_labels_classification[test_labels_classification.columns.difference(['IDs'])]
train_labels_regression = train_labels_regression[train_labels_regression.columns.difference(['IDs'])]

In [29]:
# ordering labels
train_labels_regression = train_labels_regression.reindex(sorted(train_labels_regression.columns), axis=1)
test_labels_regression = test_labels_regression.reindex(sorted(test_labels_regression.columns), axis=1)

In [30]:
print("CLASSIFICATION\n")
print("Features:")
print('Source:', train_features_source.shape, test_features_source_classification.shape)
print('Sensor:', train_features_sensor.shape, test_features_sensor_classification.shape)
print("Labels:")
print(train_labels_classification.shape, test_labels_classification.shape)

print('\n')

print("REGRESSION\n")
print("Features:")
print('Source:', train_features_source.shape, test_features_source_regression.shape)
print('Sensor:', train_features_sensor.shape, test_features_sensor_regression.shape)
print("Labels:")
print(train_labels_regression.shape, test_labels_regression.shape)


CLASSIFICATION

Features:
Source: (1440, 2790) (481, 2790)
Sensor: (1440, 4307) (481, 4307)
Labels:
(1440, 8) (481, 8)


REGRESSION

Features:
Source: (1440, 2790) (255, 2790)
Sensor: (1440, 4307) (255, 4307)
Labels:
(1440, 7) (255, 7)


In [31]:
path = os.path.join(DATA_DIR, 'split_data')

#overwriting previous save without IDs
if not(os.path.exists(path)):
    os.mkdir(path)
train_features_source.to_pickle(os.path.join(path, 'train_features_source.pkl'))
test_features_source_classification.to_pickle(os.path.join(path, 'test_features_source_classification.pkl'))
train_features_sensor.to_pickle(os.path.join(path, 'train_features_sensor.pkl'))
test_features_sensor_classification.to_pickle(os.path.join(path, 'test_features_sensor_classification.pkl'))
train_labels_classification.to_pickle(os.path.join(path, 'train_labels_classification.pkl'))
test_labels_classification.to_pickle(os.path.join(path, 'test_labels_classification.pkl'))
test_features_source_regression.to_pickle(os.path.join(path, 'test_features_source_regression.pkl'))
test_features_sensor_regression.to_pickle(os.path.join(path, 'test_features_sensor_regression.pkl'))
train_labels_regression.to_pickle(os.path.join(path, 'train_labels_regression.pkl'))
test_labels_regression.to_pickle(os.path.join(path, 'test_labels_regression.pkl'))